In [1]:
#!/usr/bin/env python
# Import packages 
import sys
import argparse as arg
# import modules in other directories
sys.path.append('../Utils/')
# import modules in other directories
sys.path.append('../Regridder/')


import importlib
import glob
import copy
import time
import os 
import subprocess as sp



import xarray as xr
import numpy as np
import pandas as pd

import ESMF as E



import scripGen as SG
import esmfRegrid as erg


# "ChatGPI version" --- 
import VertRegridFlexLL as vrg
print( "Using Flexible parallel/serial VertRegrid ")

import GridUtils as GrU
import MakePressures as MkP
import humiditycalcs as hum
import MyConstants as Con

# Reload local packages that are under
# development
importlib.reload( erg )
importlib.reload( vrg )
importlib.reload( SG )
importlib.reload( MkP )
importlib.reload( hum )
importlib.reload( GrU )
importlib.reload( Con )
#importlib.reload( Gv )


Using Flexible parallel/serial VertRegrid 


<module 'MyConstants' from '/glade/work/juliob/PyRegridding/Drivers/../Utils/MyConstants.py'>

In [2]:
####################
case = "test2.02"
BaseDir = "/glade/derecho/scratch/juliob/archive/"


#####################
Src     = 'ne30pg3'
SrcDir  = BaseDir+case+'/atm/hist/'
SrcFile = SrcDir + case + '.cam.h0.2000-01.nc'

####################
Dst     = 'fv0.9x1.25'
DstDir  = BaseDir+case+'/atm/regridded/'
DstFile = DstDir + case + '.cam.h0.2000-01.nc'


In [3]:
#######
os.makedirs( DstDir , exist_ok=True )

In [4]:
####################
if (Src == 'ne30pg3'):
    srcHkey = 'c'
    src_type='mesh'
    src_scrip = '/glade/p/cesmdata/cseg/inputdata/share/scripgrids/ne30pg3_scrip_170611.nc'
    src_TopoFile = '/glade/p/cgd/amp/juliob/bndtopo/latest/ne30pg3_gmted2010_modis_bedmachine_nc3000_Laplace0100_20230105.nc'

if ((Dst == 'fv0.9x1.25') or (Dst=='fv1x1')):
    dstHkey = 'yx'
    dst_type='grid'
    dst_scrip = '/glade/p/cesmdata/cseg/inputdata/share/scripgrids/fv0.9x1.25_141008.nc'
    dst_TopoFile = '/glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1.25_nc3000_Nsw042_Nrs008_Co060_Fi001_ZR_160505.nc'

In [5]:
# ----------------------------------------------
# Make object for ESMF regridding from SRC
# grid to CAM target. Scrip files need to be provided even 
# when a weight file is used.
# We make both conservative and bilinear mapping
# files, because we will use conservative for 2D surface vars, but
# bilinear for 3D met fields.
# ----------------------------------------------
RegridMethod = "BILINEAR"
regrdB, srcfB, dstfB = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                RegridMethod = RegridMethod )

RegridMethod = "CONSERVE"
regrdC, srcfC, dstfC = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                RegridMethod = RegridMethod )



Generating regridding weights. Method BILINEAR : ESMF method= 0
Generating regridding weights. Method CONSERVE : ESMF method= 2


In [6]:
DstTopoData  = xr.open_dataset( dst_TopoFile )

In [7]:
#####################

# Pattern to match all h0 files in the specified directory
SrcFile = SrcDir + case + '.cam.h0.*.nc'

# Get a list of all matching file paths
file_list = sorted( glob.glob(SrcFile)  )

#########################

SrcData = xr.open_dataset( file_list[0] )


#########################
lon_Dst = DstTopoData['lon'].values
lat_Dst = DstTopoData['lat'].values

slon=(lon_Dst[1:]+lon_Dst[0:-1] )/2.
slat=(lat_Dst[1:]+lat_Dst[0:-1] )/2.

lev = SrcData['lev'].values
ilev = SrcData['ilev'].values

nt,nz,ny,nx = 1,len(lev),len(lat_Dst),len(lon_Dst)
print( nt,nz,ny,nx )

dims   = ["lon","lat","time","lev","ilev","nbnd"]

# This list should be the same as the one in the ADF 
# YAML file (except PS needs to be added here)
ilist = [ 'SWCF'
        , 'LWCF'
        , 'PRECC'
        , 'PRECL'
        , 'PSL'
        , 'PS'
        , 'Q'
        , 'U'
        , 'T'
        , 'RELHUM'
        , 'TREFHT'
        , 'TS'
        , 'TAUX'
        , 'TAUY'
        , 'FSNT'
        , 'FLNT'
        , 'LANDFRAC' ]


for fileN in file_list:
    SrcFile = fileN
    DstFile = SrcFile.replace('hist','regridded')
    
    SrcData = xr.open_dataset( SrcFile )
    print( f"Read {SrcFile}")

    time = SrcData['time'].values
    timedim = SrcData['time'].dims
    
    coords = dict( 
        time = ( ["time"],  time ), #pd.to_datetime( pdTime_ERA[itim] ) ),
        lon  = ( ["lon"],lon_Dst),
        lat  = ( ["lat"],lat_Dst ),
        slon  = ( ["slon"],slon ),
        slat  = ( ["slat"],slat ),
        lev  = ( ["lev"],lev),
        ilev = ( ["ilev"],ilev),
        nbnd = ( ["nbnd"], np.array([0,1] ) ),
    )


    DstData = xr.Dataset( coords=coords  )

    DstData['time_bnds'] = SrcData['time_bnds']
    DstData['date'] = SrcData['date']
    DstData['datesec'] = SrcData['datesec']

    DstData['hyai'] = SrcData['hyai']
    DstData['hybi'] = SrcData['hybi']
    DstData['hyam'] = SrcData['hyam']
    DstData['hybm'] = SrcData['hybm']




    
    
    for fld in ilist:
        print( fld )
        shap = np.shape( SrcData[fld] ) 
        len_shap = len( shap )
        xfld_Src = SrcData[fld].values

        attrs =  SrcData[fld].attrs 

        if (len_shap == 2 ):    
            regrd = regrdC 
            srcF  = srcfC 
            dstF  = dstfC 
            xfld_Dst = np.zeros( (nt,ny,nx) , dtype=np.float64 )
            nlev=1
            dims = ('time','lat','lon',)
            Slice_Src = xfld_Src[0,:]
            Slice_Dst = erg.HorzRG( aSrc = Slice_Src , 
                         regrd = regrd , 
                         srcField= srcF , 
                         dstField= dstF , 
                         srcGridkey= srcHkey ,
                         dstGridkey= dstHkey )
            xfld_Dst[0,:,:] = Slice_Dst



        if (len_shap == 3 ):    
            regrd = regrdB 
            srcF  = srcfB
            dstF  = dstfB 
            xfld_Dst = np.zeros( (nt,nz,ny,nx) , dtype=np.float64 )
            nlev = nz
            dims = ('time','lev','lat','lon',)
            for L in np.arange( nlev ):
                Slice_Src = xfld_Src[0,L,:]
                Slice_Dst = erg.HorzRG( aSrc = Slice_Src , 
                             regrd = regrd , 
                             srcField= srcF , 
                             dstField= dstF , 
                             srcGridkey= srcHkey ,
                             dstGridkey= dstHkey )
                xfld_Dst[0,L,:,:] = Slice_Dst

        Dar = xr.DataArray( data=xfld_Dst , 
                            dims=dims,
                            attrs=attrs ,) 

        DstData[ fld ]= Dar


    DstData.to_netcdf( DstFile  , format='NETCDF3_CLASSIC'  ) #,encoding={'time': {'unlimited': True}} )
    
    
    ##sp.run(["cd "+ DstDir, "ls", "-l"] ,shell=True )
    sp.run(f" ncks --mk_rec_dmn time {DstFile} -o {DstDir}tmp.nc && \
            mv {DstDir}tmp.nc {DstFile}", shell=True )
    
    print( f"Done {DstFile}" )


1 58 192 288
Read /glade/derecho/scratch/juliob/archive/test2.02/atm/hist/test2.02.cam.h0.1995-01.nc
SWCF
LWCF
PRECC
PRECL
PSL
PS
Q
U
T
RELHUM
TREFHT
TS
TAUX
TAUY
FSNT
FLNT
LANDFRAC
Done /glade/derecho/scratch/juliob/archive/test2.02/atm/regridded/test2.02.cam.h0.1995-01.nc
Read /glade/derecho/scratch/juliob/archive/test2.02/atm/hist/test2.02.cam.h0.1995-02.nc
SWCF
LWCF
PRECC
PRECL
PSL
PS
Q
U
T
RELHUM
TREFHT
TS
TAUX
TAUY
FSNT
FLNT
LANDFRAC
Done /glade/derecho/scratch/juliob/archive/test2.02/atm/regridded/test2.02.cam.h0.1995-02.nc
Read /glade/derecho/scratch/juliob/archive/test2.02/atm/hist/test2.02.cam.h0.1995-03.nc
SWCF
LWCF
PRECC
PRECL
PSL
PS
Q
U
T
RELHUM
TREFHT
TS
TAUX
TAUY
FSNT
FLNT
LANDFRAC
Done /glade/derecho/scratch/juliob/archive/test2.02/atm/regridded/test2.02.cam.h0.1995-03.nc
Read /glade/derecho/scratch/juliob/archive/test2.02/atm/hist/test2.02.cam.h0.1995-04.nc
SWCF
LWCF
PRECC
PRECL
PSL
PS
Q
U
T
RELHUM
TREFHT
TS
TAUX
TAUY
FSNT
FLNT
LANDFRAC
Done /glade/derecho/scratch/juli

In [8]:
## OR ncks --mk_rec_dmn time_counter myfile.nc -o myfileunlimited.nc 
